In [1]:
import pandas as pd

In [54]:
Lab = pd.read_csv("FullData/LabReduced.csv", usecols = ['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc', 'UnitOfMeasure_calc'])
Exam = pd.read_csv("FullData/Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])


In [55]:
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))

dBP (mmHg)                  2570192
sBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64


In [56]:
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
Lab = Lab.loc[Lab.Name_calc.isin(tests)]
Exam = Exam.loc[Exam.Name_calc.isin(tests)]

In [57]:
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]

In [60]:
lab = pd.concat([Lab,Exam])

In [61]:
lab.isna().sum()

Patient_ID                 0
PerformedDate         432244
Name_calc                  0
TestResult_calc            0
UnitOfMeasure_calc     28971
dtype: int64

In [62]:
lab = lab.dropna(subset = ['PerformedDate'])
lab.isna().sum()

Patient_ID                0
PerformedDate             0
Name_calc                 0
TestResult_calc           0
UnitOfMeasure_calc    28660
dtype: int64

In [63]:
len(lab.index)

11993095

In [64]:
disease_periods = pd.read_csv('AP_dp.csv', index_col=None)
disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)

In [65]:
disease_periods.dtypes

Patient_ID      int64
CompOnset      object
DateOfOnset    object
dtype: object

In [66]:
disease_periods.head()

,Patient_ID,CompOnset,DateOfOnset
0,4000000526483,1992-10-05,1991-01-22
1,4000000528945,1996-05-09,1994-07-29
2,4000000527165,1998-10-08,1994-05-16
3,4000000531095,1999-06-28,1998-11-04
4,4000000610981,1999-07-05,1991-03-25


In [67]:
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]

In [68]:
print(len(patient_records.index),patient_records.columns)

350943 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc',
       'UnitOfMeasure_calc'],
      dtype='object')


In [69]:
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')

In [70]:
print(len(patient_records.index),patient_records.columns)

350943 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc',
       'UnitOfMeasure_calc', 'CompOnset', 'DateOfOnset'],
      dtype='object')


In [71]:
patient_records.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100164719,2013-10-11,Platelets in Blood (PLT),225,10^9/L,2019-07-25,2013-03-26
1,1000100164719,2013-09-22,Platelets in Blood (PLT),243,10^9/L,2019-07-25,2013-03-26
2,1000100164719,2016-02-22,Leukocytes in Blood (WBC),6.2,10^9/L,2019-07-25,2013-03-26
3,1000100164719,2016-09-12,Hemoglobin A1c (HbA1c),7.1,%,2019-07-25,2013-03-26
4,1000100164719,2013-10-11,Erythrocytes in Blood (RBC),5,10^12/L,2019-07-25,2013-03-26


In [72]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]


In [73]:
len(relVisits.index)

124009

In [81]:
relVisits.PerformedDate = pd.to_datetime(relVisits.PerformedDate,format='%Y-%m-%d')
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

In [82]:
relVisits.TestResult_calc = pd.to_numeric(relVisits.TestResult_calc,errors='coerce')
relVisits = relVisits.dropna()

In [83]:
print(relVisits.isna().sum())
print(relVisits.dtypes)

Patient_ID            0
PerformedDate         0
Name_calc             0
TestResult_calc       0
UnitOfMeasure_calc    0
CompOnset             0
DateOfOnset           0
dtype: int64
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
TestResult_calc              float64
UnitOfMeasure_calc            object
CompOnset             datetime64[ns]
DateOfOnset           datetime64[ns]
dtype: object


In [84]:
#remove vast minority units (mg/L for Total Bilirubin (n = 40) and mmol/L for Hemoglobin (n = 10))
badIndices = relVisits.loc[(relVisits.Name_calc == 'Total Bilirubin (TBIL)') & (relVisits.UnitOfMeasure_calc == 'mg/L')].index
badIndices2 = relVisits.loc[(relVisits.Name_calc == 'Hemoglobin (Hb)') & (relVisits.UnitOfMeasure_calc == 'mmol/L')].index
relVisits = relVisits.drop(badIndices)
relVisits = relVisits.drop(badIndices2)

In [85]:
Results = relVisits

In [86]:
for test in tests:
    q1 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.25)
    q3 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.75)
    rng = q3 - q1
    badIndices = Results.loc[(Results.Name_calc == test) & ((Results.TestResult_calc < (q1 - rng)) 
                                    |(Results.TestResult_calc > (q3 + rng)))].index
    Results = Results.drop(badIndices)

In [87]:
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100164719,2013-10-11,Platelets in Blood (PLT),225.0,10^9/L,2019-07-25,2013-03-26
1,1000100164719,2013-09-22,Platelets in Blood (PLT),243.0,10^9/L,2019-07-25,2013-03-26
2,1000100164719,2016-02-22,Leukocytes in Blood (WBC),6.2,10^9/L,2019-07-25,2013-03-26
3,1000100164719,2016-09-12,Hemoglobin A1c (HbA1c),7.1,%,2019-07-25,2013-03-26
4,1000100164719,2013-10-11,Erythrocytes in Blood (RBC),5.0,10^12/L,2019-07-25,2013-03-26


In [88]:
len(Results.index)

113731

In [89]:
turtiles = pd.read_csv('TestTurtiles.csv')